# Лабораторная работа №3
## Методы поиска подстроки в строке
### Выполнил студент группы БСТ1903 Минаков Л.А.

#### Задание №1

In [1]:
#Кнут-Моррис-Пратт
def prefix(t):
    v = [0]*len(t)
    for i in range(1,len(t)):
        k = v[i-1]
        while k > 0 and t[k] != t[i]:
            k = v[k-1]
        if t[k] == t[i]:
            k = k + 1
        v[i] = k
    return v

def kmp(s,t):
    """
    t - образ
    s - строка
    """
    index = -1
    f = prefix(t)
    k = 0
    for i in range(len(s)):
        while k > 0 and t[k] != s[i]:
            k = f[k-1]
        if t[k] == s[i]:
            k = k + 1
        if k == len(t):
            index = i - len(t) + 1
            break
    return index

In [2]:
#Бойера-Мура
def createShiftTable(t):
    S = set()  # уникальные символы в образе
    M = len(t) # число символов в образе
    d = {}     # словарь смещений
    
    for i in range(M-2, -1, -1): # итерации с предпоследнего символа
        if t[i] not in S:        # если символ еще не добавлен в таблицу
            d[t[i]] = M-i-1
            S.add(t[i])
    
    if t[M-1] not in S:     # отдельно формируем последний символ
        d[t[M-1]] = M
    
    d['*'] = M              # смещения для прочих символов
    return d

def bmh(s, t):
    """
    t - образ
    s - строка
    """
    N = len(s)
    M = len(t)
    d = createShiftTable(t)
    if N >= M:
        i = M-1       # счетчик проверяемого символа в строке
    
        while(i < N):
            k = 0
            j = 0
            flBreak = False
            for j in range(M-1, -1, -1):
                if s[i-k] != t[j]:
                    if j == M-1:
                        off = d[s[i]] if d.get(s[i], False) else d['*']  # смещение, если не равен последний символ образа
                    else:
                        off = d[t[j]]   # смещение, если не равен не последний символ образа
    
                    i += off    # смещение счетчика строки
                    flBreak = True  # если несовпадение символа, то flBreak = True
                    break
    
                k += 1          # смещение для сравниваемого символа в строке
    
            if not flBreak:          # если дошли до начала образа, значит, все его символы совпали
                return i-k+1
                break
    return -1

#### Стандартная функция поиска

In [3]:
def defaultSearch(s, t):
    """
    t - образ
    s - строка
    """
    return s.find(t)

#### Вывод результатов

In [4]:
import time
def search(s, t, searchType = kmp, ignoreCase = False, ignoreSpace = False):
    """
    t - образ
    s - строка
    """
    if ignoreCase:
        s = s.lower()
        t = t.lower()
    if ignoreSpace:
        s = s.replace(" ", "")
        t = t.replace(" ", "")
    startTime = time.perf_counter()
    res = searchType(s, t)
    finalTime = time.perf_counter() - startTime
    if searchType == kmp:
        stStr = "Кнута-Морриса-Пратта"
    elif searchType == bmh:
        stStr = "Упрощенный Бойера-Мура"
    else:
        stStr = "Стандартный поиск"
    print("Алгоритм поиска {}. Образ {}. Строка {}. Результат: {}. Время {}.".format(stStr, t, s, res, finalTime))
    
s = "ab cd"
t = "b c"
search(s, t, kmp)
search(s, t, bmh)
search(s, t, defaultSearch)
print()

search(s, t, kmp, ignoreSpace = False)

t = "B c"
search(s, t, kmp, ignoreCase = False)

Алгоритм поиска Кнута-Морриса-Пратта. Образ b c. Строка ab cd. Результат: 1. Время 2.4799999948754703e-05.
Алгоритм поиска Упрощенный Бойера-Мура. Образ b c. Строка ab cd. Результат: 1. Время 2.3899999973764352e-05.
Алгоритм поиска Стандартный поиск. Образ b c. Строка ab cd. Результат: 1. Время 6.600000006073969e-06.

Алгоритм поиска Кнута-Морриса-Пратта. Образ b c. Строка ab cd. Результат: 1. Время 1.3000000024021574e-05.
Алгоритм поиска Кнута-Морриса-Пратта. Образ B c. Строка ab cd. Результат: -1. Время 1.1499999970965291e-05.


#### Задание №2

In [5]:
import math
# Проверка существует ли решение
def isSolvable(board):
    sum=int(0)
    for i in range(len(board)):
        for j in range(i, len(board)):
            if board[i]==0:
                sum=sum+i//4 + 1
            elif board[j]==0:
                continue
            elif board[i]>board[j]:
                sum=sum+1
    if sum%2==0:
        return True
    return False

In [6]:
from queue import PriorityQueue

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))


class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance


SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

if not isSolvable(start):
    print('Нерешаемая позиция')
else:
    start = tuple(start)
    p = Position(start, 0)
    candidates = PriorityQueue()
    candidates.put(p)
    visited = set([p])
    
    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)
    path = []
    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

[15, 14, 10, 13, 9, 10, 14, 15]
